In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "1"

In [2]:
import csv
import re
data = []

with open('SentiNews_paragraph-level.txt', 'r') as f:
  lineReader = csv.reader(f, delimiter=',', quotechar="\"")
  for row in lineReader:
    if row:
      row = ' '.join(row)
      elementi = row.split('\t')
      sentence = elementi[2]
      sentiment = elementi[11]
      sentiment = re.sub(r'[^\w ][0-9]+', "", sentiment)
      data.append({'text': sentence, 'sent': sentiment})

In [3]:
data = data[1:]

In [4]:
import pandas as pd
df = pd.DataFrame(data, index=None, columns=['text', 'sent'])

In [5]:
df['sent'].value_counts()

neutral     40358
negative    18268
positive    10781
Name: sent, dtype: int64

In [6]:
df.sample(15)

,text,sent
55462,V pripravi sta pokojninska reforma in reforma ...,neutral
59783,Shod so preložili,neutral
27253,Tudi v evrih je cena zlata narasla. Danes ob 9...,neutral
65838,Za danes zvečer napovedani sestanek s SD so v ...,neutral
24744,Hrvaška radio televizija HRT in avstrijska ORF...,neutral
35293,Zaradi odkritja kemične snovi ki povzroča rak...,neutral
26086,Pri predstavitvi Google Eartha je bil tudi oko...,positive
67383,Ko namreč nevihtni oblaki v svojem razvoju dos...,neutral
9761,Teh podatkov nimam.,neutral
45249,Sindikat in uprava sta se dogovorila,neutral


In [7]:
df['text'] = df['text'].apply(lambda x: ' '.join(x.lower().split()))

In [8]:
df['text'] = df['text'].apply(lambda x: ' '.join(x.split()))

In [9]:
import re
df['text'] = df['text'].apply(lambda x: re.sub(r'[^\w ]+', "", x))

In [10]:
df.shape

(69407, 2)

In [11]:
df = df.dropna()

In [12]:
df.shape

(69407, 2)

In [13]:
train = df[:3669]

In [14]:
test = df[3669:5393]

In [15]:
train.shape

(3669, 2)

In [16]:
test.shape

(1724, 2)

In [17]:
#from google.colab import files
train.to_csv('sentTrainMini.csv', encoding = 'utf-8-sig', index=False) 
test.to_csv('sentTestMini.csv', encoding = 'utf-8-sig', index=False)

In [18]:
!pip install transformers datasets sklearn numpy torch torchvision

In [19]:
from datasets import load_dataset, load_metric

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [20]:
with open("sentTrainMini.csv",'r') as f:
    with open("sentTrainMiniHeadless.csv",'w') as f1:
        next(f) # skip header line
        for line in f:
          f1.write(line)

In [21]:
with open("sentTestMini.csv",'r') as f:
    with open("sentTestMiniHeadless.csv",'w') as f1:
        next(f) # skip header line
        for line in f:
          f1.write(line)

In [22]:
with open("dataParlamentS.csv",'r') as f:
    with open("dataParlamentSheadless.csv",'w') as f1:
        next(f) # skip header line
        for line in f:
            f1.write(line)

In [23]:
dataset = load_dataset(
    'csv',
    data_files={
        'train': 'sentTrainMiniHeadless.csv',
        'validation': 'dataParlamentSheadless.csv',
        'test': 'sentTestMiniHeadless.csv'
    },
    column_names = ['sentence', 'label']
)

Using custom data configuration default-5bb1e2e24cca9057


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /home/ncirar/.cache/huggingface/datasets/csv/default-5bb1e2e24cca9057/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [24]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 3669
    })
    validation: Dataset({
        features: ['sentence', 'label'],
        num_rows: 54
    })
    test: Dataset({
        features: ['sentence', 'label'],
        num_rows: 1724
    })
})

In [25]:
metric = load_metric('glue', 'sst2')

In [26]:
from transformers import AutoTokenizer

In [27]:
tokenizer = AutoTokenizer.from_pretrained(
    'EMBEDDIA/sloberta',
    use_fast=True
)

In [28]:
tokenizer(['hello', 'world'])

{'input_ids': [[5, 176, 1572, 31, 6], [5, 18810, 3714, 6]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1]]}

In [29]:
label2id = {'positive': 2, 'neutral': 1, 'negative': 0}
id2label = ['negative', 'neutral', 'positive']

In [30]:
def preprocess(examples):
  result = tokenizer(examples['sentence'], truncation=True, max_length=512)
  result['label'] = [label2id[l] for l in examples['label']]
  return result

In [31]:
encoded_dataset = dataset.map(preprocess, batched=True, load_from_cache_file=False)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [32]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'input_ids', 'attention_mask'],
        num_rows: 3669
    })
    validation: Dataset({
        features: ['sentence', 'label', 'input_ids', 'attention_mask'],
        num_rows: 54
    })
    test: Dataset({
        features: ['sentence', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1724
    })
})

In [33]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np

2022-08-28 12:05:26.468200: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [34]:
model = AutoModelForSequenceClassification.from_pretrained(
    'EMBEDDIA/sloberta',
    num_labels=3
)

Some weights of the model checkpoint at EMBEDDIA/sloberta were not used when initializing CamembertForSequenceClassification: ['lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at EMBEDDIA/sloberta and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'c

In [43]:
args = TrainingArguments(
    "tweet-sentiment-sloberta",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=6.0,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    )

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [36]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [37]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=None,
    )

In [44]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `CamembertForSequenceClassification.forward` and have been ignored: sentence. If sentence are not expected by `CamembertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 3669
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 690


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.770174,0.574074
2,No log,0.770174,0.574074
3,0.716100,0.770174,0.574074


The following columns in the evaluation set don't have a corresponding argument in `CamembertForSequenceClassification.forward` and have been ignored: sentence. If sentence are not expected by `CamembertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 54
  Batch size = 16
Saving model checkpoint to tweet-sentiment-sloberta/checkpoint-230
Configuration saved in tweet-sentiment-sloberta/checkpoint-230/config.json
Model weights saved in tweet-sentiment-sloberta/checkpoint-230/pytorch_model.bin
tokenizer config file saved in tweet-sentiment-sloberta/checkpoint-230/tokenizer_config.json
Special tokens file saved in tweet-sentiment-sloberta/checkpoint-230/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `CamembertForSequenceClassification.forward` and have been ignored: sentence. If sentence are not expected by `CamembertForSequenceClassification.forward`,  you ca

TrainOutput(global_step=690, training_loss=0.7178354899088542, metrics={'train_runtime': 152.7576, 'train_samples_per_second': 72.055, 'train_steps_per_second': 4.517, 'total_flos': 641941752663162.0, 'train_loss': 0.7178354899088542, 'epoch': 3.0})

In [45]:
eval_results = trainer.evaluate()
print(eval_results)

The following columns in the evaluation set don't have a corresponding argument in `CamembertForSequenceClassification.forward` and have been ignored: sentence. If sentence are not expected by `CamembertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 54
  Batch size = 16


{'eval_loss': 0.7701736688613892, 'eval_accuracy': 0.5740740740740741, 'eval_runtime': 0.8796, 'eval_samples_per_second': 61.39, 'eval_steps_per_second': 4.547, 'epoch': 3.0}


In [46]:
trainer.save_model(output_dir='tweet-sentiment-model-sloberta')

Saving model checkpoint to tweet-sentiment-model-sloberta
Configuration saved in tweet-sentiment-model-sloberta/config.json
Model weights saved in tweet-sentiment-model-sloberta/pytorch_model.bin
tokenizer config file saved in tweet-sentiment-model-sloberta/tokenizer_config.json
Special tokens file saved in tweet-sentiment-model-sloberta/special_tokens_map.json


In [47]:
model = AutoModelForSequenceClassification.from_pretrained('tweet-sentiment-model-sloberta')

loading configuration file tweet-sentiment-model-sloberta/config.json
Model config CamembertConfig {
  "_name_or_path": "tweet-sentiment-model-sloberta",
  "architectures": [
    "CamembertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "camembert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.21.1",
  "type_vocab_size": 1,


In [48]:
examples=['Rezultati za prejšnje leto so res pohvale vredni.',
          'Najlepša hvala za pomoč, zelo sem hvaležen.',
          'Neumni politiki nimajo pojma.', 
          'Če me ne pustiš pri miru, te bom udaril!',
          'Jutri bo deževalo.',
          'Ne maram mleka.',
          'Sovražim ponedeljke.',
          'Lansko poročilo kaže res dobre rezultate, super.',
          'Veselimo se sodelovanja z vami.',
          'Nemški ovčar je vrsta psa.']

inputs = tokenizer(examples, padding='longest', return_tensors="pt")
outputs = model(**inputs)
probs = outputs[0].detach().numpy()
for i in range(len(examples)):
    print(examples[i],'\t', id2label[np.argmax(probs[i])])

Rezultati za prejšnje leto so res pohvale vredni. 	 neutral
Najlepša hvala za pomoč, zelo sem hvaležen. 	 neutral
Neumni politiki nimajo pojma. 	 negative
Če me ne pustiš pri miru, te bom udaril! 	 neutral
Jutri bo deževalo. 	 negative
Ne maram mleka. 	 negative
Sovražim ponedeljke. 	 neutral
Lansko poročilo kaže res dobre rezultate, super. 	 neutral
Veselimo se sodelovanja z vami. 	 neutral
Nemški ovčar je vrsta psa. 	 neutral


In [35]:
import csv

corpus = []
transkript = []
dvajset = []
devetnajst = []
osemnajst = []
sedemnajst = []
sestnajst = []

i=0
with open('dataframe.csv', 'r') as f:
  lineReader = csv.reader(f, delimiter=',', quotechar="\"")
  next(f)
  for row in lineReader:
    transkript.append(row[0])
    t = row[1].split('-')
    leto = t[0]
    mesec = t[1]
    corpus.append({'text': row[0], 'leto': leto, 'mesec': mesec})
    if leto == '2020':
        dvajset.append({'text': row[0], 'leto': leto, 'mesec': mesec})
    elif leto == '2019':
        devetnajst.append({'text': row[0], 'leto': leto, 'mesec': mesec})
    elif leto == '2018':
        osemnajst.append({'text': row[0], 'leto': leto, 'mesec': mesec})
    elif leto == '2017':
        sedemnajst.append({'text': row[0], 'leto': leto, 'mesec': mesec})
    elif leto == '2016':
        sestnajst.append({'text': row[0], 'leto': leto, 'mesec': mesec})
    else:
        continue

In [36]:
import pandas as pd
df = pd.DataFrame(corpus, index=None, columns=['text', 'leto', 'mesec'])

In [37]:
df.sample(5)

,text,leto,mesec
2908,hvala gospod minister gospod danijel krivec za...,2014,12
46981,čas prosim da se držimo časovnim omejitev pred...,2017,12
31917,hvala lepa gospod franc jurša beseda je vaša i...,2016,11
57771,hvala lepa mag andrej rajh bo postavil vprašan...,2019,03
818,še kakšna obrazložitev glasu ne potem prehajam...,2014,09


In [38]:
df['leto'].value_counts()

2015    15671
2016    14724
2017    13571
2019    11877
2018     8515
2020     6601
2014     4163
Name: leto, dtype: int64

In [128]:
dva = pd.DataFrame(dvajset, index=None, columns=['text', 'leto', 'mesec'])
devet = pd.DataFrame(devetnajst, index=None, columns=['text', 'leto', 'mesec'])
osem = pd.DataFrame(osemnajst, index=None, columns=['text', 'leto', 'mesec'])
sedem = pd.DataFrame(sedemnajst, index=None, columns=['text', 'leto', 'mesec'])
sest = pd.DataFrame(sestnajst, index=None, columns=['text', 'leto', 'mesec'])

In [129]:
sest['mesec'].value_counts()

03    2176
11    2061
06    1532
12    1452
05    1445
09    1290
07    1120
04    1061
10    1049
01     794
02     744
Name: mesec, dtype: int64

In [130]:
def normalizeMonths(leto):
    jan = leto[leto['mesec']=='01']
    feb = leto[leto['mesec']=='02']
    mar = leto[leto['mesec']=='03']
    apr = leto[leto['mesec']=='04']
    maj = leto[leto['mesec']=='05']
    jun = leto[leto['mesec']=='06']
    jul = leto[leto['mesec']=='07']
    avg = leto[leto['mesec']=='08']
    sep = leto[leto['mesec']=='09']
    okt = leto[leto['mesec']=='10']
    nov = leto[leto['mesec']=='11']
    dec = leto[leto['mesec']=='12']
    
    if len(jan) >= 50:
        jan = jan.sample(50)
    if len(feb) >= 50:
        feb = feb.sample(50)
    if len(mar) >= 50:
        mar = mar.sample(50)
    if len(apr) >= 50:
        apr = apr.sample(50)
    if len(maj) >= 50:
        maj = maj.sample(50)
    if len(jun) >= 50:
        jun = jun.sample(50)
    if len(jul) >= 50:
        jul = jul.sample(50)
    if len(avg) >= 50:
        avg = avg.sample(50)
    if len(sep) >= 50:
        sep = sep.sample(50)
    if len(okt) >= 50:
        okt = okt.sample(50)
    if len(nov) >= 50:
        nov = nov.sample(50)
    if len(dec) >= 50:
        dec = dec.sample(50)
    
    meseci = [jan, feb, mar, apr, maj, jun, jul, avg, sep, okt, nov, dec]
    
    leto = pd.concat(meseci)

    return leto

In [131]:
sest = normalizeMonths(sest)
sedem = normalizeMonths(sedem)
osem = normalizeMonths(osem)
devet = normalizeMonths(devet)
dva = normalizeMonths(dva)

In [138]:
dva['mesec'].value_counts()

01    50
03    50
04    50
05    50
06    50
07    50
Name: mesec, dtype: int64

In [109]:
sest['mesec'].value_counts()

01    50
02    50
03    50
04    50
05    50
06    50
07    50
09    50
10    50
11    50
12    50
Name: mesec, dtype: int64

In [139]:
def getResults(leto):
    tekst = list(leto['text'])
    examples = tekst
    results = []
    
    inputs = tokenizer(examples, padding='longest', return_tensors="pt", max_length=512, truncation=True)
    outputs = model(**inputs)
    probs = outputs[0].detach().numpy()
    for i in range(len(examples)):
        results.append(id2label[np.argmax(probs[i])])
    
    leto['sent'] = results

In [140]:
getResults(sest)

In [141]:
sest['sent'].value_counts()

neutral     511
negative     37
positive      2
Name: sent, dtype: int64

In [142]:
getResults(sedem)
sedem['sent'].value_counts()

neutral     526
negative     23
positive      1
Name: sent, dtype: int64

In [143]:
getResults(osem)
osem['sent'].value_counts()

neutral     550
negative     38
positive      5
Name: sent, dtype: int64

In [144]:
getResults(devet)
devet['sent'].value_counts()

neutral     491
negative     26
positive     12
Name: sent, dtype: int64

In [145]:
getResults(dva)
dva['sent'].value_counts()

neutral     280
negative     18
positive      2
Name: sent, dtype: int64

In [146]:
sest.to_csv('dvaSestnajst.csv', encoding = 'utf-8-sig', index=False)

In [147]:
sedem.to_csv('dvaSedemnajst.csv', encoding = 'utf-8-sig', index=False)

In [148]:
osem.to_csv('dvaOsemnajst.csv', encoding = 'utf-8-sig', index=False)

In [149]:
devet.to_csv('dvaDevetnajst.csv', encoding = 'utf-8-sig', index=False)

In [150]:
dva.to_csv('dvaDvajset.csv', encoding = 'utf-8-sig', index=False)